In [1]:
from data_reader import Vocabulary, HWDBDatasetHelper, LMDBReader

# your path to data
train_path = r'/DATA/ichuviliaeva/ocr_data/train.lmdb'
test_path = r'/DATA/ichuviliaeva/ocr_data/test.lmdb'
gt_path = r'/DATA/ichuviliaeva/ocr_data/gt.txt'

In [26]:
import cv2
import numpy as np
from centerloss import CenterLoss
from torchvision import transforms

In [3]:
train_reader = LMDBReader(train_path)
train_reader.open()
train_helper = HWDBDatasetHelper(train_reader)

In [4]:
train_helper, val_helper = train_helper.train_val_split()

In [5]:
train_helper.size(), val_helper.size()

(2578433, 644609)

In [8]:
import torch

from torch.utils.data import Dataset, DataLoader
from torch import nn

class HWDBDataset(Dataset):
    def __init__(self, helper: HWDBDatasetHelper, transform = None):
        self.helper = helper
        self.transform = transform
    
    def __len__(self):
        return self.helper.size()
    
    def __getitem__(self, idx):
        img, label = self.helper.get_item(idx)
        if self.transform:
            img = self.transform(img)
        return img, label

In [7]:
class pad_prune(object):
    
    def __init__(self, output_w, output_h):
        self.output_w = output_w
        self.output_h = output_h

    def __call__(self, image):
        
        w, h = image.shape
        diff_w = w - self.output_w
        diff_h = h - self.output_h
        masked = np.zeros((self.output_w, self.output_h))
        if diff_w <= 0:
            if diff_h <= 0:
                masked[:w, :h] = image
            else:
                masked[:w, :] = image[:, (diff_h // 2):(h - (diff_h - diff_h // 2))]
            
        if diff_w > 0:
            if diff_h > 0:
                masked = image[(diff_w // 2):(w - (diff_w - diff_w // 2)), 
                               (diff_h // 2):(h - (diff_h - diff_h // 2))]
            else:
                masked[:, :h] = image[(diff_w // 2):(w - (diff_w - diff_w // 2)), :]
        return masked
    
class to_zero_one(object):
    def __init__(self):
        pass
    def __call__(self, image):
        return image - 127.5 / 255.0 

In [66]:
to_one_shape = pad_prune(128, 128)

train_transforms = transforms.Compose([
    to_one_shape,
    transforms.ToTensor(),
    transforms.RandomChoice([
        transforms.RandomPerspective(distortion_scale = 0.2, p = 0.9),  
        transforms.ColorJitter(),
        transforms.GaussianBlur(kernel_size = 5),
        transforms.RandomRotation(degrees=15)
    ]), 
    to_zero_one()
])

val_transforms = transforms.Compose([
    to_one_shape,
    transforms.ToTensor()
])

In [67]:
train_dataset = HWDBDataset(train_helper, train_transforms)
val_dataset = HWDBDataset(val_helper, val_transforms)

In [68]:
class ResNetBlock(nn.Module):
        
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride, is_projection = False):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(num_features = in_channels)
        self.relu1 = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, 
                               kernel_size = kernel_size, padding = padding, stride = stride)
            
        self.bn2 = nn.BatchNorm2d(num_features = out_channels)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels,
                               kernel_size = kernel_size, padding = padding, stride = stride)
        if is_projection:
            self.project = nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                               kernel_size = 1, padding = 0, stride = stride)
        else:
            self.project = lambda x: x
            
    def forward(self, x):            
        out = self.conv1(self.relu1(self.bn1(x)))
        out = self.conv2(self.relu2(self.bn2(out)))
        return out + self.project(x)

In [80]:
class TheNet(nn.Module):
    def __init__(self, n_classes):
        super(TheNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 5, stride = 2),
            nn.ReLU(),
            
            ResNetBlock(in_channels = 8, out_channels = 8, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            ResNetBlock(in_channels = 8, out_channels = 8, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
           
            ResNetBlock(in_channels = 16, out_channels = 16, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            ResNetBlock(in_channels = 16, out_channels = 16, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
            
            ResNetBlock(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1, stride = 1,
                             is_projection = True),
            ResNetBlock(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
            
            ResNetBlock(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1, stride = 1,
                             is_projection = True),
            ResNetBlock(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2)),
            
            ResNetBlock(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            ResNetBlock(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1, stride = 1, 
                             is_projection = True),
            
            nn.Flatten(),
            nn.Linear(3 * 3 * 128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        
        self.last_layer = nn.Linear(256, n_classes, bias=False)
        
    def forward(self, x):
        x = self.nn(x)
        y = self.last_layer(x)
        return x, y

In [81]:
model = TheNet(train_helper.vocabulary.num_classes())
model.eval()
res = model(torch.tensor(train_dataset[0][0], dtype=torch.float32).view(1, 1, 128, 128))
print(res[0].shape)

torch.Size([1, 256])


/tmp/ipykernel_298014/1968819161.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = model(torch.tensor(train_dataset[0][0], dtype=torch.float32).view(1, 1, 128, 128))


In [82]:
model = model.cuda()

In [83]:
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False, num_workers=8)

In [84]:
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [85]:
center_loss = CenterLoss(num_classes=train_helper.vocabulary.num_classes(), feat_dim=256, use_gpu=True)
optimizer_centloss = torch.optim.Adam(center_loss.parameters(), lr=0.5)

In [86]:
from tqdm.notebook import tqdm

def run_validation(val_loader: DataLoader, model: nn.Module, n_steps=None):
    model.eval()
    n_good = 0
    n_all = 0
    wrapper = lambda x: x
    if n_steps is None:
        n_steps = len(val_loader)
        wrapper = tqdm
    
    with torch.no_grad():
        for batch, (X, y) in enumerate(wrapper(val_loader)):
            if batch == n_steps:
                break
            features, logits = model(X.to(torch.float32).cuda())
            classes = torch.argmax(logits, dim=1).cpu().numpy()
            n_good += sum(classes == y.cpu().numpy())
            n_all += len(classes)
    
    return n_good / n_all


def train_epoch(train_loader: DataLoader, val_loader: DataLoader, model: nn.Module, optim, loss_fn, alpha):
    for batch, (X, y) in enumerate(tqdm(train_loader)):
        model.train()
        features, logits = model(X.to(torch.float32).cuda())
        labels = y.to(torch.long).cuda()
        loss = center_loss(features, labels) * alpha + loss_fn(logits, labels)
        
        optim.zero_grad()
        optimizer_centloss.zero_grad()
        
        loss.backward()
        optim.step()
        
        for param in center_loss.parameters():
             param.grad.data *= (1./alpha)

        optimizer_centloss.step()

In [87]:
test_path = r'/DATA/ichuviliaeva/ocr_data/test.lmdb'
pred_path = './pred.txt'

test_reader = LMDBReader(test_path)
test_reader.open()
test_helper = HWDBDatasetHelper(test_reader, prefix='Test')

In [88]:
test_dataset = HWDBDataset(test_helper, to_one_shape)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False, num_workers=8)

In [90]:
from pathlib import Path

def evaluate(gt_path, pred_path):
    gt = dict()
    with open(gt_path) as gt_f:
        for line in gt_f:
            name, cls = line.strip().split()
            gt[name] = cls
    
    n_good = 0
    n_all = len(gt)
    with open(pred_path) as pred_f:
        for line in pred_f:
            name, cls = line.strip().split()
            if cls == gt[name]:
                n_good += 1
    
    return n_good / n_all


def _run_evaluation():
    base = '.'
    pred_path = base + '/pred.txt'
    print('pred_path = ', pred_path)
    score = evaluate(gt_path, pred_path)
    print('Accuracy = {:1.4f}'.format(score))

Тут я просто захотела периодически запускать тест в середине. Это не должно приводить к каким-либо утечкам данных, т. к. всё предсказывается под with torch.no_grad()

In [91]:
alpha = 0.3
for epoch in range(31):
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, val_loader, model, optim, loss_fn, alpha)
    accuracy = run_validation(val_loader, model)
    print(f'accuracy: {accuracy}')
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'arch1_all_{epoch}.pth')
        alpha = min(1.0, alpha * 1.5)
        
        preds = []
        model.eval()
        with torch.no_grad():
            for X, _ in tqdm(test_loader):
                features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
                classes = torch.argmax(logits, dim=1).cpu().numpy()
                preds.extend(classes)
                
        with open(pred_path, 'w') as f_pred:
            for idx, pred in enumerate(preds):
                name = test_helper.namelist[idx]
                cls = train_helper.vocabulary.class_by_index(pred)
                print(name, cls, file=f_pred)
        _run_evaluation()

Epoch 0:


  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.004962698317895034


  0%|          | 0/380 [00:00<?, ?it/s]

...

  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.9092224899124896


  0%|          | 0/380 [00:00<?, ?it/s]

pred_path =  ./pred.txt
Accuracy = 0.8749


In [92]:
epoch = 31
while epoch < 61:
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, val_loader, model, optim, loss_fn, alpha)
    accuracy = run_validation(val_loader, model)
    print(f'accuracy: {accuracy}')
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'arch1_all_{epoch}.pth')
        alpha = min(1.0, alpha * 1.5)
        
        preds = []
        model.eval()
        with torch.no_grad():
            for X, _ in tqdm(test_loader):
                features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
                classes = torch.argmax(logits, dim=1).cpu().numpy()
                preds.extend(classes)
                
        with open(pred_path, 'w') as f_pred:
            for idx, pred in enumerate(preds):
                name = test_helper.namelist[idx]
                cls = train_helper.vocabulary.class_by_index(pred)
                print(name, cls, file=f_pred)
        _run_evaluation()
    epoch += 1

Epoch 31:


  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.9119729944819263
Epoch 32:


  0%|          | 0/5036 [00:00<?, ?it/s]

...

  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.9290856938081845


  0%|          | 0/380 [00:00<?, ?it/s]

pred_path =  ./pred.txt
Accuracy = 0.9022


In [93]:
epoch = 61
while epoch < 81:
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, val_loader, model, optim, loss_fn, 1.2)
    accuracy = run_validation(val_loader, model)
    print(f'accuracy: {accuracy}')
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'arch1_all_{epoch}.pth')
        
        preds = []
        model.eval()
        with torch.no_grad():
            for X, _ in tqdm(test_loader):
                features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
                classes = torch.argmax(logits, dim=1).cpu().numpy()
                preds.extend(classes)
                
        with open(pred_path, 'w') as f_pred:
            for idx, pred in enumerate(preds):
                name = test_helper.namelist[idx]
                cls = train_helper.vocabulary.class_by_index(pred)
                print(name, cls, file=f_pred)
        _run_evaluation()
    epoch += 1

Epoch 61:


  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.932161977260634
Epoch 62:


  0%|          | 0/5036 [00:00<?, ?it/s]

...

  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.9495275430532307


  0%|          | 0/380 [00:00<?, ?it/s]

pred_path =  ./pred.txt
Accuracy = 0.9296


In [100]:
epoch = 81
while epoch < 91:
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, val_loader, model, optim, loss_fn, 1.2)
    accuracy = run_validation(val_loader, model)
    print(f'accuracy: {accuracy}')
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'arch1_all_{epoch}.pth')
        
        preds = []
        model.eval()
        with torch.no_grad():
            for X, _ in tqdm(test_loader):
                features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
                classes = torch.argmax(logits, dim=1).cpu().numpy()
                preds.extend(classes)
                
        with open(pred_path, 'w') as f_pred:
            for idx, pred in enumerate(preds):
                name = test_helper.namelist[idx]
                cls = train_helper.vocabulary.class_by_index(pred)
                print(name, cls, file=f_pred)
        _run_evaluation()
    epoch += 1

Epoch 81:


  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.9501511769149981
Epoch 82:


  0%|          | 0/5036 [00:00<?, ?it/s]

...

  0%|          | 0/5036 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

accuracy: 0.9522904582467822


  0%|          | 0/380 [00:00<?, ?it/s]

pred_path =  ./pred.txt
Accuracy = 0.9299


In [101]:
model.load_state_dict(torch.load(f'arch1_all_85.pth'))

<All keys matched successfully>

In [102]:
test_path = r'/DATA/ichuviliaeva/ocr_data/test.lmdb'
pred_path = './pred.txt'

test_reader = LMDBReader(test_path)
test_reader.open()
test_helper = HWDBDatasetHelper(test_reader, prefix='Test')

In [103]:
test_dataset = HWDBDataset(test_helper, to_one_shape)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False, num_workers=8)

In [49]:
# model.load_state_dict(torch.load(f'arch1_epoch25.pth'))

<All keys matched successfully>

In [104]:
preds = []
model.eval()
with torch.no_grad():
    for X, _ in tqdm(test_loader):
        features, logits = model(X.unsqueeze(1).to(torch.float32).cuda())
        classes = torch.argmax(logits, dim=1).cpu().numpy()
        preds.extend(classes)

  0%|          | 0/380 [00:00<?, ?it/s]

In [105]:
with open(pred_path, 'w') as f_pred:
    for idx, pred in enumerate(preds):
        name = test_helper.namelist[idx]
        cls = train_helper.vocabulary.class_by_index(pred)
        print(name, cls, file=f_pred)

In [106]:
!python -m course_ocr_t2.evaluate
# Accuracy = 0.7978

pred_path =  /home/ichuviliaeva/ocr_course/course_ocr/task2/pred.txt
Accuracy = 0.9324
